In [5]:
import cv2
import numpy as np
import os
import sys
sys.path.append(os.path.abspath('../'))
import pandas as pd
import datetime
import random
import utilities.helper_functions as hf
import config

In [6]:
video_path = config.remote_video_path
processed_data_path = config.successful_trial_path
reference_path = [config.marker_path + dir for dir in os.listdir(config.marker_path)]

In [7]:
# get list of video names from the folder of processed CSVs
csv_times = [ hf.process_time_string(name.split('_')[0]) for name in os.listdir(processed_data_path)]
csv_paths = [name for name in os.listdir(processed_data_path)]
# iterate through each csv and find the closest video name
video_times = [hf.process_time_string(name.split('_')[0]) for name in os.listdir(video_path)]
video_paths = [name for name in os.listdir(video_path)]
csv_vid_match = dict()
for csv_time in csv_times:
    closest_time = video_times[0]
    closest_path = video_paths[0]
    for i, video_time in enumerate(video_times):
        if abs((csv_time - video_time).total_seconds()) <  abs((csv_time - closest_time).total_seconds()):
            closest_time = video_time
            closest_path = video_paths[i]
    total_delta = (csv_time - closest_time).total_seconds()
    if total_delta < 10:
        csv_vid_match[csv_paths[csv_times.index(csv_time)]] = {
            'video_name': closest_path,
            'delta_seconds': total_delta
        }



significant_trials = []
for csv_name in csv_vid_match.keys():
    df = pd.read_csv(processed_data_path + '/' + csv_name)
    if len(df['successful_trials']) >= 10:
        significant_trials.append(csv_name)
significant_trials.sort()
significant_trials.reverse()
# randoize order
#random.shuffle(significant_trials)


    

In [8]:
for test_path in significant_trials:
    test_video = cv2.VideoCapture(video_path + csv_vid_match[test_path]['video_name'])
    test_video = hf.scrub_video_seconds(test_video, 10)
    # get single frame
    success, image = test_video.read()
    # boost contrast
    window_name = test_path.split('/')[-1]
    cv2.imshow(window_name, image)
    # create slider to adjust contrast
    def nothing(x):
        pass
    # create bars to crop image
    cv2.createTrackbar('x1', window_name, 0, 1000, nothing)
    cv2.createTrackbar('y1', window_name, 0, 1000, nothing)
    cv2.createTrackbar('x2', window_name, 1, 1000, nothing)
    cv2.createTrackbar('y2', window_name, 1, 1000, nothing)
    cv2.setTrackbarPos('x2', window_name, 200)
    cv2.setTrackbarPos('y2', window_name, 200)
    while True:
        # crop newimage 
        x1 = cv2.getTrackbarPos('x1', window_name)
        y1 = cv2.getTrackbarPos('y1', window_name)
        x2 = cv2.getTrackbarPos('x2', window_name)
        y2 = cv2.getTrackbarPos('y2', window_name)
        new_image = image[y1:y2, x1:x2]
        cv2.imshow('frame', new_image)
        cv2.imshow(window_name, image)

        if cv2.waitKey(1) & 0xFF == ord('q'):
            break    
        if cv2.waitKey(1) & 0xFF == ord('s'):
            # read through 1000 frames of the video
            for i in range(10000):
                success, image = test_video.read()
            continue
    cv2.destroyAllWindows()
    # save the image to a file
    # display the image
    # check if the image path exists
    if not os.path.exists(config.marker_path  + '/interaction_area_' + test_path.split('.')[0] + '.jpg'):
        cv2.imwrite(config.marker_path  + '/interaction_area_' + test_path.split('.')[0] + '.jpg', new_image)
    else:
        cv2.imwrite(config.marker_path  + '/interaction_area_' + test_path.split('.')[0] + str(random.randint(0, 100)) + '.jpg', new_image)

KeyboardInterrupt: 